# Note about file saving
When run completely this notebook will generate 3 files: 2 .json and 1 .csv. `my_test_data.json` gets written over every run; it's just a placeholder, you shouldn't need it for anything.

These files will be saved in the same directory as the notebook. You can move the files after you run the whole notebook. Just don't move the .jsons anywhere until you've made the .csv, unless you want to alter the file paths in the last cell.

In [1]:
# Scrape imports
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener
from datetime import datetime
import shutil

# Processing imports
import pandas as pd
import json

# Scraping
#### This first cell is all you need to alter.
Punch in your keys and put the terms you want to track in the to_track list. Don't forget your quotes.

In [2]:
###### Authentication and Tracking settings

consumer_key = 'bmHm3j3uPr843UAxhAFkVWy5K'
consumer_secret = 'cRc0dFKxP3fcBOy4W4dhZHE70BkMXVqgryFR3DcA7MWdIoJMGL'
access_token = '1034804083515961344-xWIE6Mq6VynHUVSdMF2nmdXTxw3b7w'
access_secret = 'EXOYf0OPG6ID8DCE0wdml4T0H4qmghne5l6nx1FBGYH0R'

to_track = ['#ItsATestItsATest']

In [3]:
# Tweepy authentication stuff I don't understand
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)

# This class is necessary to pulling in tweets live.
# Source: https://marcobonzanini.com/2015/03/02/mining-twitter-data-with-python-part-1/
class MyListener(StreamListener):
    def on_data(self, data):
        try:
            with open('scrape.json', 'a') as f:
                f.write(data)
                return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True
 
    def on_error(self, status):
        print(status)
        return True

## Scraping Cell
### Stopping:
- You need to kernel -> interrupt every time you want it to stop.
- It will still save any tweets it's already collected when you interrupt it.

### Saving:
- The data gets autosaved into a unique json file every time this cell runs.
- If you don't want a billion json files...try to not run this cell unnecessarily.
- (You might also get bumped off twitter if you run it too much idk.)

### Last note:
- Remember to change what you're tracking if you're tracking different stuff.
- I included a commented repeat of the to_track definition so you can change it here if you want (instead of the first cell).

In [4]:
################## Scraping cell

#to_track = ['#HurricaneFlorence']
twitter_stream = Stream(auth, MyListener())
twitter_stream.filter(track=to_track)

KeyboardInterrupt: 

In [5]:
###### File rename
# This cell will rename the scrape.json file you just created to something unique,
# so you aren't writing over the same file every time and losing access to previous data.
# (if you do want to keep writing over 'scrape.json' then you can skip running this cell).

now = datetime.now()
if now.month < 10:
    month = '0'+str(now.month)
else: month = str(now.month)
if now.day < 10:
    day = '0'+str(now.day)
else: day = str(now.day)
if now.hour < 10:
    hour = '0'+str(now.hour)
else: hour = str(now.hour)
if now.minute < 10:
    minute = '0'+str(now.minute)
else: minute = str(now.minute)
if now.second < 10:
    second = '0'+str(now.second)
else: second = str(now.second)

now_str = str(now.year)+month+day+'_'+hour+minute+second
dest = './tweepy_scrape_'+now_str+'.json'
shutil.move('./scrape.json', dest)

'./tweepy_scrape_20181002_144309.json'

# Exporting to csv
#### If you are running this immediately after scraping then nothing here needs altering, just run the cell.

In [6]:
input_filepath = dest

# Read the entire file into a python array
with open(input_filepath, 'r') as f:
    data = f.readlines()

jsons = []
for ind in data:
    jsons.append(json.loads(ind))

# Read into json
with open('my_test_data.json', 'w') as outfile:
    json.dump(jsons, outfile)
    
# Create a pandas series of the json data
json_df = pd.read_json('my_test_data.json',typ='series')

####################### Convert the series into list of dictionaries ##########################
dict_list = []
for row in json_df:
    temp_dict = {}
    
    # Basic tweet data: ID and timestamp
    temp_dict['id'] = row['id']
    temp_dict['created_at'] = row['created_at']
    
    try:
        temp_dict['url'] = row['entities']['urls'][0]['url']
    except:
        # Recreate url using username and tweet ID
        user = row['user']['screen_name']
        url = 'https://twitter.com/'+user+'/status/'+row['id_str']
        temp_dict['url'] = url
    
    # Tweet location data, if available
    temp_dict['geo'] = row['geo']
    temp_dict['coords'] = row['coordinates']
    
    # Retrieve tweet text
    try:
        temp_dict['text'] = row['extended_tweet']['full_text']
    except:
        temp_dict['text'] = row['text']
    
    # Try to retrieve image url, if available
    try:
        temp_dict['image_url'] = row['extended_tweet']['entities']['media'][0]['media_url']
    except:
        temp_dict['image_url'] = None
    
    # Get user data
    temp_dict['user_id'] = row['user']['id']
    temp_dict['user_name'] = row['user']['name']
    temp_dict['user_screen_name'] = row['user']['screen_name']
    temp_dict['user_location'] = row['user']['location']
    temp_dict['user_geoenabled'] = row['user']['geo_enabled']
    
    # Append temp dict to dict_list
    dict_list.append(temp_dict)
############################################################################################
    
# Create a dataframe from the list of dictionaries
data = pd.DataFrame(dict_list,index=list(range(len(dict_list))))

filename = 'processed_tweepy_'+now_str+'.csv'

# Write to csv
data.to_csv(filename)
pd.read_csv(filename)     # Just checking that it worked

,Unnamed: 0,coords,created_at,geo,id,image_url,text,url,user_geoenabled,user_id,user_location,user_name,user_screen_name
0,0,NaN,Tue Oct 02 18:42:35 +0000 2018,NaN,1047195154799583234,NaN,#ItsATestItsATest hooray,https://twitter.com/aprilg091/status/104719515...,False,1034804083515961344,NaN,April,aprilg091
1,1,NaN,Tue Oct 02 18:42:45 +0000 2018,NaN,1047195194821697536,NaN,#ItsATestItsATest yippee,https://twitter.com/aprilg091/status/104719519...,False,1034804083515961344,NaN,April,aprilg091
2,2,NaN,Tue Oct 02 18:42:55 +0000 2018,NaN,1047195237599404032,NaN,#ItsATestItsATest the world is greattttt,https://twitter.com/aprilg091/status/104719523...,False,1034804083515961344,NaN,April,aprilg091
